# ----------------------------------------------------------------
# SCREEN TIME RECOMMENDATION SYSTEM 
# ----------------------------------------------------------------


In [1]:
import pandas as pd
import numpy as np


In [2]:
# STEP 1: Load Dataset
# --------------------------------------------
df = pd.read_csv("Indian_Kids_Screen_Time_Cleaned.csv")

In [3]:
# Calculate Educational and Recreational screen time
df['Educational_Avg_Screen_Time_hr'] = df['Avg_Daily_Screen_Time_hr'] / (1 + df['Educational_to_Recreational_Ratio'])
df['Recreational_Avg_Screen_Time_hr'] = df['Avg_Daily_Screen_Time_hr'] - df['Educational_Avg_Screen_Time_hr']

# To round off to 2 decimal places (optional)
df['Educational_Avg_Screen_Time_hr'] = df['Educational_Avg_Screen_Time_hr'].round(2)
df['Recreational_Avg_Screen_Time_hr'] = df['Recreational_Avg_Screen_Time_hr'].round(2)

print("Dataset loaded successfully with shape:", df.shape)

Dataset loaded successfully with shape: (9668, 10)


In [4]:
# STEP 2: Health Impact Insights
# --------------------------------------------
df['Health_Impacts'] = (
    df['Health_Impacts']
    .astype(str)
    .str.replace(r"[\[\]']", '', regex=True)   # remove brackets and quotes
    .str.replace('"', '', regex=False)          # remove extra quotes
    .str.strip()                                # strip outer spaces
    .str.split(',')                             # split by comma
)

df_exploded = df.explode('Health_Impacts')
df_exploded['Health_Impacts'] = df_exploded['Health_Impacts'].str.strip() #split comma separated value to a new row

# Average screen time per impact
avg_screen_time_per_impact = (
    df_exploded.groupby('Health_Impacts')['Avg_Daily_Screen_Time_hr']
    .mean()
    .reset_index()
    .sort_values(by='Avg_Daily_Screen_Time_hr', ascending=False)
)

print("\n📊 Average Screen Time per Health Impact:\n", avg_screen_time_per_impact)


📊 Average Screen Time per Health Impact:
   Health_Impacts  Avg_Daily_Screen_Time_hr
4     Poor Sleep                  4.931915
3   Obesity Risk                  4.924979
0        Anxiety                  4.911559
1     Eye Strain                  4.871348
2      No Impact                  3.245264


In [5]:
# STEP 3: Age, Device, and Gender Insights
# --------------------------------------------
non_exceeded = df[df['Exceeded_Recommended_Limit'] == False]

# Average screen time per age among non-exceeded users
avg_screen_time_age = (
    non_exceeded.groupby('Age')['Avg_Daily_Screen_Time_hr']
    .mean()
    .reset_index()
    .rename(columns={'Avg_Daily_Screen_Time_hr': 'Avg_Screen_Time_By_Age'})
)

# Average screen time per device among non-exceeded users
avg_screen_time_device = (
    non_exceeded.groupby('Primary_Device')['Avg_Daily_Screen_Time_hr']
    .mean()
    .reset_index()
    .rename(columns={'Avg_Daily_Screen_Time_hr': 'Avg_Screen_Time_By_Device'})
)

# Average screen time per gender among non-exceeded users
avg_screen_time_gender = (
    non_exceeded.groupby('Gender')['Avg_Daily_Screen_Time_hr']
    .mean()
    .reset_index()
    .rename(columns={'Avg_Daily_Screen_Time_hr': 'Avg_Screen_Time_By_Gender'})
)

# Convert to dictionaries for lookup
age_to_limit = dict(zip(avg_screen_time_age['Age'], avg_screen_time_age['Avg_Screen_Time_By_Age']))
device_to_limit = dict(zip(avg_screen_time_device['Primary_Device'], avg_screen_time_device['Avg_Screen_Time_By_Device']))
gender_to_limit = dict(zip(avg_screen_time_gender['Gender'], avg_screen_time_gender['Avg_Screen_Time_By_Gender']))

In [6]:
# STEP 4: Helper Functions
# --------------------------------------------
def get_age_based_limit(age):
    return round(age_to_limit.get(age, df['Avg_Daily_Screen_Time_hr'].mean()), 2)

def get_device_based_limit(device):
    return round(device_to_limit.get(device, df['Avg_Daily_Screen_Time_hr'].mean()), 2)

def get_gender_based_limit(gender):
    return round(gender_to_limit.get(gender, df['Avg_Daily_Screen_Time_hr'].mean()), 2)

def get_combined_recommended_limit(age, device, gender):
    combined_limit = (
        (get_age_based_limit(age) + get_device_based_limit(device) + get_gender_based_limit(gender)) / 3
    )
    return round(combined_limit, 2)

def exceeded_logic(age, device, gender, screen_time):
    age_limit = get_age_based_limit(age)
    device_limit = get_device_based_limit(device)
    gender_limit = get_gender_based_limit(gender)
    combined_limit = get_combined_recommended_limit(age, device, gender)
    return (
        screen_time > age_limit,
        screen_time > device_limit,
        screen_time > gender_limit,
        screen_time > combined_limit
    )

In [7]:
# STEP 5: Main Recommendation System
# --------------------------------------------
def recommendation_system(age, gender, device, screen_time):
    users_behind = round((df['Avg_Daily_Screen_Time_hr'] < screen_time).mean() * 100, 2)

    age_limit = get_age_based_limit(age)
    device_limit = get_device_based_limit(device)
    gender_limit = get_gender_based_limit(gender)
    combined_limit = get_combined_recommended_limit(age, device, gender)

    exceeded_age, exceeded_device, exceeded_gender, exceeded_combined = exceeded_logic(age, device, gender, screen_time)

    return {
        "Age": age,
        "Gender": gender,
        "Device": device,
        "Your Screen Time (hrs)": screen_time,
        "Age-based Recommended Limit (hrs)": age_limit,
        "Device-based Recommended Limit (hrs)": device_limit,
        "Gender-based Recommended Limit (hrs)": gender_limit,
        "Combined Recommended Limit (hrs)": combined_limit,
        "Exceeded Age Limit": exceeded_age,
        "Exceeded Device Limit": exceeded_device,
        "Exceeded Gender Limit": exceeded_gender,
        "Exceeded Combined Limit": exceeded_combined,
        "Users having lower Screen Time (%)": users_behind
    }

In [8]:
# STEP 6: User Input
# --------------------------------------------
print("\nWelcome to the Enhanced Screen Time Recommendation System ⚡")
user_age = int(input("Enter your age: "))
user_gender = input("Enter your gender (Male/Female): ").capitalize()
user_device = input("Enter your primary device (Smartphone/Laptop/TV etc.): ").capitalize()
user_screen_time = float(input("Enter your average daily screen time (in hours): "))

# STEP 7: Educational vs Recreational Health Analysis
# --------------------------------------------

print("\nLet's analyze your screen time split 👀")

# User inputs total & educational screen time only
user_total_time = float(input("Enter your average daily TOTAL screen time (in hours): "))
user_edu_time = float(input("Enter your average daily EDUCATIONAL screen time (in hours): "))

# Automatically calculate recreational time
user_rec_time = max(user_total_time - user_edu_time, 0)  # ensure no negatives

print(f"\n📘 Educational Time: {user_edu_time:.2f} hrs")
print(f"🎮 Recreational Time (auto-calculated): {user_rec_time:.2f} hrs")

# Ensure limits exist
if 'edu_limit' not in locals() or 'rec_limit' not in locals():
    non_exceeded = df[df['Exceeded_Recommended_Limit'] == False]
    edu_limit = round(non_exceeded['Educational_Avg_Screen_Time_hr'].mean(), 2)
    rec_limit = round(non_exceeded['Recreational_Avg_Screen_Time_hr'].mean(), 2)
    print(f"\n📘 Recommended Educational Limit (hrs): {edu_limit}")
    print(f"🎮 Recommended Recreational Limit (hrs): {rec_limit}")

# 🧠 EDUCATIONAL OVERUSE ANALYSIS
if user_edu_time > edu_limit:
    edu_window = df_exploded[
        (df_exploded['Avg_Daily_Screen_Time_hr'] >= user_edu_time - 0.5) &
        (df_exploded['Avg_Daily_Screen_Time_hr'] <= user_edu_time + 0.5)
    ]
    impacts_edu = edu_window['Health_Impacts'].dropna().astype(str).str.strip().unique()
    impacts_edu = sorted(set([i.title() for i in impacts_edu if i.lower() != 'no impact']))

    if impacts_edu:
        print("\n🧠 Health Impacts (Educational Overuse):")
        for impact in impacts_edu:
            print(f" - {impact}")
    else:
        print("\n🧠 Educational Overuse Detected, but no strong negative health impacts recorded near this range.")
else:
    print("\n✅ Educational Screen Time is within recommended limit.")

# 🎮 RECREATIONAL OVERUSE ANALYSIS
if user_rec_time > rec_limit:
    rec_window = df_exploded[
        (df_exploded['Avg_Daily_Screen_Time_hr'] >= user_rec_time - 0.5) &
        (df_exploded['Avg_Daily_Screen_Time_hr'] <= user_rec_time + 0.5)
    ]
    impacts_rec = rec_window['Health_Impacts'].dropna().astype(str).str.strip().unique()
    impacts_rec = sorted(set([i.title() for i in impacts_rec if i.lower() != 'no impact']))

    if impacts_rec:
        print("\n🎮 Health Impacts (Recreational Overuse):")
        for impact in impacts_rec:
            print(f" - {impact}")
    else:
        print("\n🎮 Recreational Overuse Detected, but no strong health impacts found around this range.")
else:
    print("\n✅ Recreational Screen Time is within recommended limit.")

    
# STEP 8: Generate Recommendation Report
# --------------------------------------------
result = recommendation_system(user_age, user_gender, user_device, user_screen_time)

print("\n--- RECOMMENDATION REPORT ---")
for k, v in result.items():
    print(f"{k}: {v}")

diff = result['Your Screen Time (hrs)'] - result['Combined Recommended Limit (hrs)']
if diff > 0:
    print(f"\n⚠️ You are spending {diff:.2f} hrs more than your personalized combined limit.")
else:
    print(f"\n✅ Great! You are {abs(diff):.2f} hrs below your personalized recommended limit.")



Welcome to the Enhanced Screen Time Recommendation System ⚡
Enter your age: 18
Enter your gender (Male/Female): Female
Enter your primary device (Smartphone/Laptop/TV etc.): Smartphone
Enter your average daily screen time (in hours): 4.01

Let's analyze your screen time split 👀
Enter your average daily TOTAL screen time (in hours): 4.01
Enter your average daily EDUCATIONAL screen time (in hours): 2.8

📘 Educational Time: 2.80 hrs
🎮 Recreational Time (auto-calculated): 1.21 hrs

📘 Recommended Educational Limit (hrs): 1.14
🎮 Recommended Recreational Limit (hrs): 0.48

🧠 Health Impacts (Educational Overuse):
 - Anxiety
 - Eye Strain
 - Obesity Risk
 - Poor Sleep

🎮 Recreational Overuse Detected, but no strong health impacts found around this range.

--- RECOMMENDATION REPORT ---
Age: 18
Gender: Female
Device: Smartphone
Your Screen Time (hrs): 4.01
Age-based Recommended Limit (hrs): 2.48
Device-based Recommended Limit (hrs): 1.72
Gender-based Recommended Limit (hrs): 1.64
Combined Recomm

In [9]:
max_combined_limit = 0
best_combo = (None, None, None)

# Iterate through all combinations of age, device, and gender
for age in age_to_limit.keys():
    for device in device_to_limit.keys():
        for gender in gender_to_limit.keys():
            combined_limit = (
                (age_to_limit[age] + device_to_limit[device] + gender_to_limit[gender]) / 3
            )
            if combined_limit > max_combined_limit:
                max_combined_limit = combined_limit
                best_combo = (age, device, gender)

print("\n🌟 Combination with Maximum Combined Limit 🌟")
print(f"Age: {best_combo[0]}")
print(f"Device: {best_combo[1]}")
print(f"Gender: {best_combo[2]}")
print(f"Maximum Combined Limit (hrs): {round(max_combined_limit, 2)}")



🌟 Combination with Maximum Combined Limit 🌟
Age: 18
Device: Laptop
Gender: Female
Maximum Combined Limit (hrs): 2.15
